In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fpdf import FPDF
import shutil
from IPython.display import display

# ==============================================================================
# 1. CONFIGURAÇÃO INICIAL
# ==============================================================================

# --- Definir caminhos ---
output_dir = '../../analises/'
temp_img_dir = os.path.join(output_dir, 'temp_images/')

# --- Criar diretórios se não existirem ---
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_img_dir, exist_ok=True)

In [15]:
# ==============================================================================
# 2. CARREGAMENTO DOS DADOS
# ==============================================================================

# Caminho para os dados tratados
caminho = '../../dados_tratados/csv/'

# Carregando os dados em um dicionário de DataFrames
dados = {
    'alunos': pd.read_csv(caminho + 'escolas_alunos_2006.csv', sep=';', encoding='utf-8'),
    'professores': pd.read_csv(caminho + 'escolas_professores_2006.csv', sep=';', encoding='utf-8'),
}

print("Dados de Alunos (antes do tratamento):")
display(dados['alunos'].head())



Dados de Alunos (antes do tratamento):


,nome_escola,cidade_escola,local_bh_rmbh,bairro_escola,turma,hora_inicio,hora_fim,aplicadores,observacoes,data_aplicacao,...,participacao_programa,participacao_pais,pertence_gangue,envolvimento_gangues,ja_furtou,uso_bebida,uso_cigarro,uso_solventes,uso_maconha,uso_crack
0,e.e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia e nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,escola estadual celso machado,belo horizonte,bh,milionarios,2,36600,38700.0,adenir,NaN,13362192384,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0
2,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0
3,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery e klarissa,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,e. e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia/nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [16]:
# ==============================================================================
# 3. PRÉ-PROCESSAMENTO DOS DADOS
# ==============================================================================

# --- Dicionários para renomear colunas ---
rename_dict_alunos = {
    'sexo': 'sexo',
    'idade': 'idade',
    'raca_cor': 'raca_cor',
    'qual_outra_raca': 'raca_cor_aux' # Coluna auxiliar
}

rename_dict_professores = {
    'tempo_como_professor_na_escola': 'tempo_como_professor_na_escola',
    'materia': 'materia',
    'percep_seguranca': 'percepcao_seguranca'
}

# --- Processar Alunos ---
# Padroniza colunas para caixa baixa
dados['alunos'].columns = dados['alunos'].columns.str.lower().str.strip()
dados['alunos'] = dados['alunos'].rename(columns=rename_dict_alunos)

# Tratamento específico da coluna 'raca_cor'
if 'raca_cor' in dados['alunos'].columns and 'raca_cor_aux' in dados['alunos'].columns:
    mask_outra = dados['alunos']['raca_cor'] == 'outra cor ou raça'
    dados['alunos'].loc[mask_outra, 'raca_cor'] = dados['alunos'].loc[mask_outra, 'raca_cor_aux']
    
    # Excluir registros com 'uma outra mistura'
    dados['alunos'] = dados['alunos'][dados['alunos']['raca_cor'] != 'uma outra mistura']
    
    # Remover colunas auxiliares
    dados['alunos'] = dados['alunos'].drop(columns=['raca_cor_aux'])

# --- Processar Professores ---
dados['professores'].columns = dados['professores'].columns.str.lower().str.strip()
dados['professores'] = dados['professores'].rename(columns=rename_dict_professores)

print("\nPré-processamento concluído.")
print("Dados dos Alunos Tratados:")
display(dados['alunos'].head())
print("\nDados dos Professores Tratados:")
display(dados['professores'].head())




Pré-processamento concluído.
Dados dos Alunos Tratados:


,nome_escola,cidade_escola,local_bh_rmbh,bairro_escola,turma,hora_inicio,hora_fim,aplicadores,observacoes,data_aplicacao,...,participacao_programa,participacao_pais,pertence_gangue,envolvimento_gangues,ja_furtou,uso_bebida,uso_cigarro,uso_solventes,uso_maconha,uso_crack
0,e.e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia e nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,escola estadual celso machado,belo horizonte,bh,milionarios,2,36600,38700.0,adenir,NaN,13362192384,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0
2,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0
3,e. e. ari da franca,belo horizonte,bh,santa monica,6,50640,53520.0,nery e klarissa,NaN,13366598656,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,e. e. geraldina ana gomes,belo horizonte,bh,jardim europa,7,36240,38760.0,lidia/nery,NaN,13364697088,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0



Dados dos Professores Tratados:


,nome_escola,cidade_escola,bairro_escola,turma,hora_inicio,hora_fim,aplicadores,observacoes,data_aplicacao,id_questionario,...,pergunta_generica_q4,"se sim, você poderia citar algum desses objetivos",participa_programa_escola_viva,descricao_participacao_programa,pergunta_q51,pergunta_q51_quais.1,percepcao_participacao_externa_programa,pergunta_q53,opiniao_programa_escola_viva,digitador
0,margarida de melo prado,bh,monte azul,1,67140.0,67800.0,nery,NaN,1.336668e+10,54,...,0.0,"integracao maior entre escola e comunidade, al...",0.0,na,0.0,na,0.0,0.0,0.0,rodrigo heringer costa
1,e. e. cecilia meireles,bh,teixeira dias,3,69600.0,71100.0,nr,NaN,1.336539e+10,44,...,0.0,defender a escola de individuos que nao deveri...,0.0,na,0.0,na,0.0,0.0,0.0,rodrigo heringer costa
2,e. e. prof. ligia maria magalhaes,contagem,colorado,8,37200.0,0.0,NaN,NaN,1.336539e+10,45,...,0.0,"maior participacao, intercambio(escola-comunid...",0.0,"participacao de reunioes, debate para a melhor...",0.0,na,0.0,0.0,0.0,rodrigo heringer costa
3,e. e. joao guimaraes rosa,betim,icaivera,6,36240.0,37920.0,nery,NaN,0.000000e+00,50,...,0.0,unir escola e comunidade,0.0,na,0.0,na,0.0,0.0,0.0,rodrigo heringer costa
4,margarida de melo prado,bh,monte azul,8,66600.0,68400.0,keli,NaN,1.336660e+10,53,...,0.0,na,0.0,na,0.0,na,0.0,0.0,0.0,rodrigo heringer costa


In [ ]:
# ==============================================================================
# 4. ANÁLISE E GERAÇÃO DE GRÁFICOS
# ==============================================================================

sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (10, 6)

pdf_content = [] # Lista para armazenar informações para o PDF

# --- Análise de Alunos ---

# Gráfico 1: Distribuição de Alunos por Sexo
plt.figure()
ax = sns.countplot(data=dados['alunos'], x='sexo', order=dados['alunos']['sexo'].value_counts().index)
ax.set_title('Distribuição de Alunos por Sexo', fontsize=16)
ax.set_xlabel('Sexo', fontsize=12)
ax.set_ylabel('Quantidade de Alunos', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(temp_img_dir, 'alunos_sexo.png'))
plt.close()
pdf_content.append({
    'title': 'Análise de Alunos: Gênero',
    'image': 'alunos_sexo.png',
    'analysis': "O gráfico mostra a distribuição de alunos por sexo. A análise permite identificar a proporção de estudantes masculinos e femininos, fornecendo um panorama demográfico inicial."
})

# Gráfico 2: Distribuição de Alunos por Raça/Cor
plt.figure()
ax = sns.countplot(data=dados['alunos'], y='raca_cor', order=dados['alunos']['raca_cor'].value_counts().index)
ax.set_title('Distribuição de Alunos por Raça/Cor', fontsize=16)
ax.set_xlabel('Quantidade de Alunos', fontsize=12)
ax.set_ylabel('Raça/Cor', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(temp_img_dir, 'alunos_raca.png'))
plt.close()
pdf_content.append({
    'title': 'Análise de Alunos: Raça/Cor',
    'image': 'alunos_raca.png',
    'analysis': "Este gráfico detalha a diversidade racial entre os alunos. Após o tratamento dos dados, a distribuição se torna mais clara, permitindo avaliar a representatividade de diferentes grupos étnico-raciais na comunidade escolar."
})

# --- Análise de Professores ---

# Gráfico 3: Anos de Experiência dos Professores
plt.figure()
ax = sns.histplot(data=dados['professores'], x='tempo_como_professor_na_escola', bins=5, kde=True)
ax.set_title('Distribuição de Anos de Experiência dos Professores', fontsize=16)
ax.set_xlabel('tempo como professor na escola', fontsize=12)
ax.set_ylabel('Quantidade de Professores', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(temp_img_dir, 'prof_experiencia.png'))
plt.close()
pdf_content.append({
    'title': 'Análise de Professores: Experiência',
    'image': 'prof_experiencia.png',
    'analysis': "A análise da experiência docente revela a senioridade do corpo de professores. O histograma mostra se a maioria dos professores está no início de carreira ou possui vasta experiência, um fator que pode influenciar a qualidade do ensino."
})

# Gráfico 4: Percepção de Segurança na Escola
plt.figure()
ax = sns.countplot(data=dados['professores'], x='percepcao_seguranca', order=dados['professores']['percepcao_seguranca'].value_counts().index)
ax.set_title('Percepção de Segurança na Escola pelos Professores', fontsize=16)
ax.set_xlabel('Percepção de Segurança', fontsize=12)
ax.set_ylabel('Quantidade de Professores', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(temp_img_dir, 'prof_seguranca.png'))
plt.close()
pdf_content.append({
    'title': 'Análise de Professores: Segurança',
    'image': 'prof_seguranca.png',
    'analysis': "Este gráfico ilustra como os professores percebem a segurança no ambiente escolar. É um indicador crucial do clima organizacional e do bem-estar do corpo docente, podendo impactar diretamente a motivação e o desempenho."
})

print("\nGráficos de análise gerados e salvos com sucesso.")


KeyError: 'percepcao_seguranca'

<Figure size 1000x600 with 0 Axes>

In [ ]:
# ==============================================================================
# 5. EXPORTAÇÃO DO RELATÓRIO CONSOLIDADO EM PDF
# ==============================================================================

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Relatório Consolidado - Análise de Escolas 2006', 0, 1, 'C')
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_analysis_section(self, title, image_path, analysis_text):
        self.add_page()
        self.chapter_title(title)
        
        if os.path.exists(image_path):
            self.image(image_path, x=30, w=150)
            self.ln(5)
        else:
            self.set_font('Arial', 'I', 10)
            self.cell(0, 10, f"[Imagem não encontrada em '{image_path}']", 0, 1)
        
        self.chapter_body(analysis_text)

# --- Gerar o PDF ---
pdf = PDF()
pdf_filename = os.path.join(output_dir, 'relatorio_final_escolas_2006.pdf')

for item in pdf_content:
    full_image_path = os.path.join(temp_img_dir, item['image'])
    pdf.add_analysis_section(item['title'], full_image_path, item['analysis'])

pdf.output(pdf_filename)
print(f"\nPDF '{pdf_filename}' gerado com sucesso.")


# ==============================================================================
# 6. LIMPEZA DOS ARQUIVOS TEMPORÁRIOS
# ==============================================================================
try:
    shutil.rmtree(temp_img_dir)
    print(f"Pasta temporária '{temp_img_dir}' foi apagada com sucesso.")
except OSError as e:
    print(f"Erro ao apagar a pasta temporária: {e}")